In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

%run sudoku_solver.ipynb

In [2]:
#Preprocessing image
img = cv2.imread('opencv-assets/sudoku-puzzle.jfif')
img = cv2.resize(img, (450, 450))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(img_gray, (5,5), 0)
img_thres = cv2.adaptiveThreshold(img_blur, 255, 1, 1, 11, 2)

cv2.imshow('Preprocessed image', img_thres)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#Contours detection
img_copy = img.copy()
contours, _ = cv2.findContours(img_thres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img_copy, contours, -1, (0,0,255), 2)

cv2.imshow('Contours', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
#Function to detect the biggest contour within image and return vertices in order
def biggest_contour_in_order(contours):
    
    largest = np.array([])
    max_area = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        polygon = cv2.approxPolyDP(cnt, 0.02*perimeter, True)
        if area > max_area and len(polygon) == 4:
            max_area = area
            largest = polygon
    
    biggest = np.empty((4,1,2), dtype=np.int32)
    largest = largest.reshape((4,2))
    
    row_sum = np.sum(largest, axis=1)
    biggest[0] = largest[np.argmin(row_sum)]
    biggest[3] = largest[np.argmax(row_sum)]
    
    row_diff = np.diff(largest, axis=1)
    biggest[1] = largest[np.argmin(row_diff)]
    biggest[2] = largest[np.argmax(row_diff)]
    
    return biggest


big = biggest_contour_in_order(contours)

In [5]:
img_big_contour = img.copy()
img_big_contour = cv2.drawContours(img_big_contour, big, -1, (0,255,0), 10)

cv2.imshow('Approximated contour', img_big_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
pts1 = np.float32(big)
pts2 = np.float32([[0,0], [450,0], [0,450], [450,450]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
img_warp_color = cv2.warpPerspective(img, matrix, (450,450))

cv2.imshow('Warped', img_warp_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
#Load model
new_model = keras.models.load_model('dr_model.h5')

img_warp_gray = cv2.cvtColor(img_warp_color, cv2.COLOR_BGR2GRAY)
img_warp_gray = cv2.equalizeHist(img_warp_gray)

#Split the image into 81 equal parts
nums = []
rows = np.vsplit(img_warp_gray, 9)
for row in rows:
    col = np.hsplit(row, 9)
    for cell in col:
        nums.append(cell)

In [8]:
#Finds the number of pixels closer to black in an image
def no_of_pixels(img):
    count = 0
    rows = img.shape[0]
    cols = img.shape[1]
    
    for i in range(rows):
        for j in range(cols):
            if(img[i][j] < 100):
                count = count + 1
                
    return count

In [9]:
board = []
binary_board = []

for num in nums:
    pic = np.asarray(num)
    pic = pic[3:pic.shape[0]-3, 3:pic.shape[1]-3]
    
    if no_of_pixels(pic) < 300:
        board.append(0)
        binary_board.append(0)
        continue
    
    pic = pic/255
    pic = cv2.resize(pic, (28,28))
    pic = pic.reshape(-1,28,28,1)
    
    result = new_model.predict(pic)
    digit = np.argmax(result)
    
    board.append(digit)
    binary_board.append(1)
    
board = np.array(board)
binary_board = np.array(binary_board)
board = board.reshape(9,9)
binary_board = binary_board.reshape(9,9)

In [10]:
grid = np.zeros((9,9))
print(board)

if solve(board):
    grid = board
else:
    print("Prediction error or no solution :(")

[[3 0 0 8 0 1 0 0 2]
 [2 0 1 0 3 0 6 0 4]
 [0 0 0 2 0 4 0 0 0]
 [8 0 9 0 0 0 1 0 6]
 [0 6 0 0 0 0 0 5 0]
 [7 0 2 0 0 0 4 0 9]
 [0 0 0 5 0 9 0 0 0]
 [9 0 4 0 8 0 7 0 5]
 [6 0 0 1 0 7 0 0 3]]


In [11]:
def print_on_board(img, row, col, num):
    
    cell_side = img.shape[0]//9
    offset = 10
    orig = (col*cell_side+offset, cell_side*row+cell_side-offset)
    cv2.putText(img, str(num), orig, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


for i in range(9):
    for j in range(9):
        if binary_board[i][j] == 0:
            print_on_board(img_warp_color, i, j, grid[i][j])
            

In [12]:
from PIL import Image

img_new = img.copy()
img_new = cv2.cvtColor(img_new, cv2.COLOR_BGR2RGB)
pil_img_new = Image.fromarray(img_new)

big = np.asarray(big)
big = big.reshape(4,2)
width = big[1][0]-big[0][0]
height = big[2][1]-big[0][1]
img_warp_color = cv2.resize(img_warp_color, (width, height))
img_warp_color = cv2.cvtColor(img_warp_color, cv2.COLOR_BGR2RGB)
pil_img_warp_color = Image.fromarray(img_warp_color)

pil_img_new.paste(pil_img_warp_color, (big[0][0], big[0][1]))

In [13]:
img_new = cv2.cvtColor(np.asarray(pil_img_new), cv2.COLOR_RGB2BGR)
cv2.imshow('Final image', img_new)
cv2.waitKey(0)
cv2.destroyAllWindows()